## 자물쇠와 열쇠
https://school.programmers.co.kr/learn/courses/30/lessons/60059

In [ ]:
def rotate(arr, d):
    n = len(arr)
    result = [[0] * n for _ in range(n)]

    if d % 4 == 1:
        for r in range(n):
            for c in range(n):
                result[c][n - r - 1] = arr[r][c]

    elif d % 4 == 2:
        for r in range(n):
            for c in range(n):
                result[n - r - 1][n - c - 1] = arr[r][c]

    elif d % 4 == 3:
        for r in range(n):
            for c in range(n):
                result[n - c - 1][r] = arr[r][c]
    else:
        for r in range(n):
            for c in range(n):
                result[r][c] = arr[r][c]
    return result

def check(lock):
    n = len(lock) // 3
    for i in range(n, n*2):
        for j in range(n, n*2):
            if lock[i][j] != 1:
                return False
    return True


def solution(key, lock):
    m = len(key)
    n = len(lock)
    new_lock = [[0] * (n*3) for _ in range(n*3)]

    for i in range(n):
        for j in range(n):
            new_lock[n+i][n+j] = lock[i][j]

    for i in range(1, n*2):
        for j in range(1, n*2):
            for d in range(4):
                r_key = rotate(key, d)
                for x in range(m):
                    for y in range(m):
                        new_lock[i + x][j + y] += r_key[x][y]
                if check(new_lock):
                    return True
                for x in range(m):
                    for y in range(m):
                        new_lock[i + x][j + y] -= r_key[x][y]
    return False


print(solution([[0, 0, 0], [1, 0, 0], [0, 1, 1]], [[1, 1, 1], [1, 1, 0], [1, 0, 1]]))

## 셔틀버스
https://school.programmers.co.kr/learn/courses/30/lessons/17678

In [ ]:
from datetime import datetime, timedelta

def solution(n, t, m, timetable):
    answer = 0
    inter = timedelta(minutes=t)
    bus = [datetime.strptime("09:00", "%H:%M") + inter * i for i in range(n)]

    timeSort = []
    for time in timetable:
        timeSort.append(datetime.strptime(time, '%H:%M'))
    timeSort.sort()

    i = 0
    for bm in bus:
        cnt = 0
        while cnt < m and i < len(timeSort) and timeSort[i] <= bm:
            i += 1
            cnt += 1
        if cnt < m:
            answer = bm
        else:
            answer = timeSort[i-1] - timedelta(minutes=1)

    return datetime.strftime(answer, '%H:%M')

In [ ]:
def solution(n, t, m, timetable):
    crewTime = sorted([int(t[:2]) * 60 + int(t[3:]) for t in timetable])
    busTime = [9*60 + i*t for i in range(n)]
    answer = 0

    i = 0
    for bt in busTime:
        cnt = 0
        while cnt < m and i < len(crewTime) and crewTime[i] <= bt:
            i += 1
            cnt += 1
        if cnt < m:
            answer = bt
        else:
            answer = crewTime[i-1] - 1

    return str(answer//60).zfill(2) + ":" + str(answer%60).zfill(2)


solution(2, 10, 2, ["09:10", "09:09", "08:00"])

## 표 편집
https://school.programmers.co.kr/learn/courses/30/lessons/81303

In [ ]:
# 틀림

def solution(n, k, cmd):
    delete = []
    total = n
    for c in cmd:
        if c[0] == 'D':
            k += int(c[2:])
        elif c[0] == 'U':
            k -= int(c[2:])
        elif c[0] == 'C':
            delete.append(k)
            if total-1 == k:
                k -= 1
            if k == 0:
                k += 1
            total -= 1
        else:
            z = delete.pop()
            if k <= z:
                k += 1
            total += 1
    ans = ['O'] * n
    for d in sorted(delete):
        ans[d] = 'X'
    return ''.join(ans)

print(solution(8, 2, ["D 2","C","U 3","C","D 4","C","U 2","Z","Z"]))


In [ ]:
def solution(n, k, cmd):
    cur = k
    table = {i: [i-1, i+1] for i in range(n)}
    answer = ['O'] * n
    table[0] = [None, 1]
    table[n-1] = [n-2, None]

    stack = []
    for c in cmd:
        if c == "C":
            answer[cur] = 'X'
            prev, nxt = table[cur]
            stack.append([prev, cur, nxt])
            if nxt == None:
                cur = table[cur][0]
            else:
                cur = table[cur][1]
            
            if prev == None:
                table[nxt][0] = None
            elif nxt == None:
                table[prev][1] = None
            else:
                table[prev][1] = nxt
                table[nxt][0] = prev

        elif c == "Z":
            prev, now, nxt = stack.pop()
            answer[now] = 'O'
            if prev == None:
                table[nxt][0] = now
            elif nxt == None:
                table[prev][1] = now
            else:
                table[prev][1] = now
                table[nxt][0] = now
        else:
            c1, c2 = c.split()
            c2 = int(c2)
            if c1 == 'D':
                for _ in range(c2):
                    cur = table[cur][1]
            else:
                for _ in range(c2):
                    cur = table[cur][0]

    return ''.join(answer)


print(solution(8, 2, ["D 2","C","U 3","C","D 4","C","U 2","Z","Z"]))

## 등산코스
https://school.programmers.co.kr/learn/courses/30/lessons/118669

In [ ]:
from collections import defaultdict
from heapq import heappop, heappush


# n: 노드 수
# gates: 출입구, sumits: 산봉우리
def solution(n, paths, gates, summits):
    
    def get_min_intensity():
        pq = []
        visited = [1e9] * (n+1)

        for gate in gates:
            heappush(pq, (0, gate))
            visited[gate] = 0
        
        while pq:
            intensity, node = heappop(pq)
            if node in summits or intensity > visited[node]:
                continue
            
            for weight, next_node in graph[node]:
                new_intensity = max(intensity, weight)
                if new_intensity < visited[next_node]:
                    visited[next_node] = new_intensity
                    heappush(pq, (new_intensity, next_node))

        min_intensity = [0, 1e9]
        for summit in summits:
            if visited[summit] < min_intensity[1]:
                min_intensity[0] = summit
                min_intensity[1] = visited[summit]
        return min_intensity

    summits.sort()

    graph = defaultdict(list)
    for path in paths:
        i, j, w = path
        graph[i].append((w, j))
        graph[j].append((w, i))

    return get_min_intensity()

## 코딩 테스트 공부
https://school.programmers.co.kr/learn/courses/30/lessons/118668

In [3]:
def solution(alp, cop, problems):
    max_alp, max_cop = 0, 0

    for a, b, c, d, e in problems:
        max_alp = max(max_alp, a)
        max_cop = max(max_cop, b)
    alp = min(max_alp, alp)
    cop = min(max_cop, cop)

    dp = [[1e9] * (max_cop + 1) for _ in range(max_alp + 1)]
    dp[alp][cop] = 0

    for i in range(alp, max_alp + 1):
        for j in range(cop, max_cop + 1):
            if i+1 <= max_alp:
                dp[i+1][j] = min(dp[i+1][j], dp[i][j] + 1)
            if j+1 <= max_cop:
                dp[i][j+1] = min(dp[i][j+1], dp[i][j] + 1)
            for alp_req, cop_req, alp_rwd, cop_rwd, cost in problems:
                if alp_req <= i and cop_req <= j:
                    next_alp, next_cop = min(max_alp, i + alp_rwd), min(max_cop, j + cop_rwd)
                    dp[next_alp][next_cop] = min(dp[next_alp][next_cop], dp[i][j] + cost)
    return dp[-1][-1]
print(solution(10, 10, [[10,15,2,1,2],[20,20,3,3,4]]))

15
